In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
import glob

from tifffile import imread
from csbdeep.utils import Path, download_and_extract_zip_file, plot_some
from csbdeep.io import save_tiff_imagej_compatible
from csbdeep.models import CARE

/data/u934/service_imagerie/v_kapoor/anaconda2/envs/tensorflowpy3pt5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/u934/service_imagerie/v_kapoor/anaconda2/envs/tensorflowpy3pt5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/u934/service_imagerie/v_kapoor/anaconda2/envs/tensorflowpy3pt5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [5]:
basedir = '/data/u934/service_imagerie/v_kapoor/StarDistData/TestData/'
basedirResultsUnet = '/data/u934/service_imagerie/v_kapoor/StarDistData/JuliaResultsData/Unet/'

ModelName= 'UNet_model'

BaseDirModels = 'models'

In [6]:
Path = os.path.join(basedir, '*.tif')
X = []
Names = []

filesRaw = glob.glob(Path)

for fname in filesRaw:
     x = imread(fname)
     X.append(x)
     Names.append(fname)
X.sort

SegmentedLabelsWithoutShape = [];
SegmentedLabelsWithShape = [];
axes = 'YX'
print('Total number of images to be segmented = ', len(X))
print('Image size = ', X[1].shape)
count = min(0, len(X))

Total number of images to be segmented =  3
Image size =  (512, 640)


In [8]:
model = CARE(config = None, name = ModelName, basedir = BaseDirModels)

Loading network weights from 'weights_best.h5'.


In [19]:
%%time
Segmentedfiles = []
for x in X:
    segmented = model.predict(x, axes)
    Segmentedfiles.append(segmented)

/data/u934/service_imagerie/v_kapoor/anaconda2/envs/tensorflowpy3pt5/lib/python3.5/site-packages/csbdeep/utils/utils.py:209: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  x = x[slices]
/data/u934/service_imagerie/v_kapoor/anaconda2/envs/tensorflowpy3pt5/lib/python3.5/site-packages/csbdeep/data/prepare.py:284: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[crop]


CPU times: user 409 ms, sys: 209 ms, total: 618 ms
Wall time: 563 ms


In [21]:

for i in range(0, len(Names)):
   print('Saving file' +  basedirResultsUnet + '%s_' + os.path.basename(Names[i]))
   save_tiff_imagej_compatible((basedirResultsUnet + '%s_' + os.path.basename(Names[i])) % model.name, Segmentedfiles[i], axes)

Saving file/data/u934/service_imagerie/v_kapoor/StarDistData/JuliaResultsData/Unet/%s_calibrated-P51.001.tif
Saving file/data/u934/service_imagerie/v_kapoor/StarDistData/JuliaResultsData/Unet/%s_calibrated-P51.003.tif
Saving file/data/u934/service_imagerie/v_kapoor/StarDistData/JuliaResultsData/Unet/%s_calibrated-P51.002.tif


/data/u934/service_imagerie/v_kapoor/anaconda2/envs/tensorflowpy3pt5/lib/python3.5/site-packages/csbdeep/utils/utils.py:209: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  x = x[slices]


In [23]:
for  i  in  range(0,  count):
      plt.figure(figsize=(16,10))
      plot_some(np.stack([Segmentedfiles[i]]),
          title_list=[['U-Net Segmentation']], 
          pmin=2,pmax=99.8);
      print('Filename = ', Names[i])